In [1]:
from openai import OpenAI
import os
import re

In [2]:
os.environ["DASHSCOPE_API_KEY"] = "sk-2ea9416b45e04af6b6aa72d3c2ade52f"

In [3]:
# 定义路径
parent_dir = "生成结果"
sub_dir = "social_media"
output_dir = os.path.join(parent_dir, sub_dir)

# 先确保父目录存在
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)
    print(f"已创建目录: {parent_dir}")

# 检查子目录是否存在，如果不存在则创建
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"已创建子目录: {output_dir}")
else:
    print(f"子目录已存在: {output_dir}")

prompt_file_path = 'Data/提示词.md'
product_type_file_path = 'Data/产品类别.txt'
output_file = os.path.join(output_dir, '理论假设.md')

# 读取产品类别文件内容
try:
    with open(product_type_file_path, 'r', encoding='utf-8') as file:
        product_type = file.read().strip()
    print(f"成功读取产品类别: {product_type}")
except FileNotFoundError:
    print(f"错误：找不到产品类别文件 {product_type_file_path}")
    raise
except Exception as e:
    print(f"读取产品类别文件时出错: {e}")
    raise


子目录已存在: 生成结果/social_media


In [4]:
# 读取提示词模板文件内容
try:
    with open(prompt_file_path, 'r', encoding='utf-8') as file:
        prompt_template = file.read().strip()
    
    # 替换模板中的占位符
    prompt_content = prompt_template.replace("{product_type}", product_type)
    
    # 添加额外的指示，避免重复
    prompt_content += "\n\n【重要提示】：请只生成一个表格，不要重复之前的内容。请提供一个全新的、完整的表格，而不是添加到之前的内容后面。"
    
    print("成功处理提示词模板")
except FileNotFoundError:
    print(f"错误：找不到提示词模板文件 {prompt_file_path}")
    raise
except Exception as e:
    print(f"处理提示词模板时出错: {e}")
    raise

# 初始化OpenAI客户端
try:
    client = OpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    print("成功初始化API客户端")
except Exception as e:
    print(f"初始化API客户端时出错: {e}")
    raise


In [5]:
# 存储返回内容
reasoning_content = ""
answer_content = ""
is_answering = False

try:
    # 创建聊天完成请求
    completion = client.chat.completions.create(
        model="qwq-plus",
        messages=[
            {"role": "user", "content": prompt_content}
        ],
        stream=True
    )
    
    print("\n" + "=" * 20 + "正在处理" + "=" * 20 + "\n")
    
    for chunk in completion:
        if not chunk.choices:
            if hasattr(chunk, 'usage'):
                print("\nToken使用情况:")
                print(chunk.usage)
        else:
            delta = chunk.choices[0].delta
            # 收集思考过程
            if hasattr(delta, 'reasoning_content') and delta.reasoning_content is not None:
                print(".", end='', flush=True)  # 显示处理进度
                reasoning_content += delta.reasoning_content
            else:
                # 收集回复内容
                if delta.content != "" and is_answering is False:
                    print("\n开始生成回复...")
                    is_answering = True
                if hasattr(delta, 'content') and delta.content is not None:
                    answer_content += delta.content
    
    # 处理重复表格问题
    # 方法1：如果存在多个markdown表格，只保留最后一个
    markdown_tables = re.findall(r'(\|[\s\S]+?\n\n)', answer_content)
    if len(markdown_tables) > 1:
        print(f"\n检测到 {len(markdown_tables)} 个表格，只保留最后一个")
        answer_content = markdown_tables[-1]
    
    # 方法2：如果以上方法无效，尝试匹配完整的表格格式
    if not markdown_tables:
        markdown_tables = re.findall(r'(\|\s*序号\s*\|[\s\S]+?)\n\n(?=\||\Z)', answer_content)
        if len(markdown_tables) > 1:
            print(f"\n检测到 {len(markdown_tables)} 个表格，只保留最后一个")
            answer_content = markdown_tables[-1]
    
    # 将结果保存到output_dir/hypo.md
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(answer_content)
    
    print("\n\n" + "=" * 20 + "处理完成" + "=" * 20)
    print(f"结果已保存到 {output_file}")
    
except Exception as e:
    print(f"\n处理过程中出错: {e}")


====================正在处理====================

.................................................................................................
开始生成回复...


====================处理完成====================
结果已保存到 生成结果/social_media/理论假设.md
